In [2]:
import os
import glob
import numpy as np
import pandas as pd

import pyvista as pv

from geotransformer.utils.pointcloud import apply_transform

In [3]:
def load_from_bin(bin_path):
    obj = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    return obj[:,:3]

def load_from_npy(npy_path):
    obj = np.load(npy_path)
    return obj

In [4]:
# Load Transforms
transforms = np.load('transforms.npy')
gt_transforms = pd.read_csv('data/Rellis/sequences/00/poses.txt', header=None, sep=' ')

# Get all the bin files (Point Clouds)
bin_files = sorted(glob.glob(os.path.join('data/Rellis/sequences/04/os1_cloud_node_kitti_bin/', '*.bin')))
npy_files = sorted(glob.glob(os.path.join('data/Rellis/downsampled/04/', '*.npy')))

In [5]:
start = 25
end = 100
skip = 50


point_cloud = load_from_bin(bin_files[start])
for i, bin_file in enumerate(bin_files[start+skip:end:skip]):
    idx = i * skip + start
    # Load Point Cloud
    src_point_cloud = load_from_bin(bin_file)
    
    # Get New Transform
    gt_transform = np.concatenate([gt_transforms.iloc[idx].to_numpy().reshape(3,4), np.array([[0,0,0,1]])], axis=0)
    
    # transform = np.matmul(transform, gt_transform)
    transformed_point_cloud = apply_transform(src_point_cloud, gt_transform)
    
    os.makedirs('./reg_pointclouds/gt', exist_ok=True)
    np.save('./reg_pointclouds/gt/{}.npy'.format(i), transformed_point_cloud)
    
    # Apply Transform
    point_cloud = np.append(point_cloud, transformed_point_cloud, axis=0)#apply_transform(src_point_cloud, transform))


point_cloud = load_from_npy(npy_files[start])
transform = np.eye(4)
for i, npy_file in enumerate(npy_files[start+skip:end:skip]):
    idx = i * skip + start
    
    # Load Point Cloud
    src_point_cloud = load_from_npy(npy_file)
    
    # Get New Transform
    pred_transform = transforms[i]
    
    transform = np.matmul(pred_transform, transform)
    transformed_point_cloud = apply_transform(src_point_cloud, transform)
    
    os.makedirs('./reg_pointclouds/pred', exist_ok=True)
    np.save('./reg_pointclouds/pred/{}.npy'.format(i), transformed_point_cloud)
    
    # Apply Transform
    point_cloud = np.append(point_cloud, transformed_point_cloud, axis=0)#apply_transform(src_point_cloud, transform))
    

In [6]:
pv.start_xvfb()
point_cloud_pv = pv.PolyData(point_cloud)
point_cloud_pv.plot(eye_dome_lighting=True, render_points_as_spheres=True, jupyter_backend='static')